In [1]:
import os
import json
import gc
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from transformers import AutoTokenizer
from typing import Dict, List, Set, Tuple, NamedTuple, Callable
import scipy
import scml
from scml import pandasx as pdx

In [2]:
tim = scml.Timer()
tim.start()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()
info = np.iinfo(np.int16)
print(f"int16, min={info.min}, max={info.max}")

int16, min=-32768, max=32767


In [3]:
tokenizer = AutoTokenizer.from_pretrained("huggingface/microsoft/deberta-v3-base", is_fast=True)
print(f"{tokenizer}\nmodel_input_names={tokenizer.model_input_names}")

DebertaV2TokenizerFast(name_or_path='huggingface/microsoft/deberta-v3-base', vocab_size=128000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	128000: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
model_input_names=['input_ids', '

/home/pankun/.local/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [4]:
sdf = pd.read_csv("input/persuade20/sources.csv", low_memory=False)
sdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   prompt_name   12 non-null     object
 1   title         12 non-null     object
 2   author        12 non-null     object
 3   url           12 non-null     object
 4   gpt4_summary  9 non-null      object
dtypes: object(5)
memory usage: 608.0+ bytes


In [5]:
sdf.head(12)

,prompt_name,title,author,url,gpt4_summary
0,Car-free cities,"In German Suburb, Life Goes On Without Cars",Elisabeth Rosenthal,https://www.nytimes.com/2009/05/12/science/earth/12suburb.html,"Vauban, an experimental district near Freiburg, Germany, is pioneering suburban living without reliance on cars. Established in 2006, it prohibits street parking, driveways, and home garages, allowing car ownership but limiting parking to large garages on the development's edge, costing $40,000 per space. This innovative concept has led 70% of Vauban’s families to live without cars, with 57% selling their cars to move there. Emphasizing “smart planning,” Vauban is a model in reducing greenhouse gas emissions, a critical factor in combating global warming. The district, home to 5,500 residents, showcases how suburban areas can be more compact, accessible to public transportation, and less car-dependent. This approach aligns with efforts in the U.S. and Europe to develop suburbs that favor public transport and walking, with amenities within walking distance. However, despite these efforts, challenges remain in changing car-centric attitudes and adapting urban planning regulations to support such sustainable living models."
1,Car-free cities,Paris bans driving due to smog,Robert Duffer,https://scoringguides.cambiumast.com/pdfs/Grade_10.pdf,"Paris faced severe pollution, prompting a partial driving ban. Motorists with even-numbered plates were barred on Monday, facing a 22-euro fine, with odd-numbered plates targeted the next day. Nearly 4,000 fines were issued, and 27 cars were impounded. This action reduced congestion by 60 percent, addressing smog levels comparable to Beijing. The smog, intensified by weather conditions, was partly blamed on diesel vehicles, which are prevalent in France due to tax policies. Paris generally experiences more smog than other European capitals, with significantly higher particulate matter levels than cities like Brussels or London. The driving restrictions led to complaints from delivery companies but exceptions were made for eco-friendly and high-occupancy vehicles. Public transport was free from Friday to Monday. The improved air quality by Monday led to the lifting of the ban for Tuesday."
2,Car-free cities,Car-free day is spinning into a big hit in Bogota,Andrew Selsky,https://scoringguides.cambiumast.com/pdfs/Grade_10.pdf,"In Bogotá, Colombia, a successful ""Day Without Cars"" was observed for the third consecutive year, aimed at encouraging alternative transport and reducing air pollution. This initiative, where millions left their cars at home and opted for hiking, biking, skating, or public transportation, has become a remarkable event in this city of 7 million. Despite occasional rain, participation was high, with fines for violations set at $25. Bogotá Mayor Antanas Mockus highlighted the event's significance in reducing stress and air pollution. This year, for the first time, the Colombian cities of Cali and Valledupar also participated, drawing interest from international municipal authorities, like Enrique Riera, mayor of Asunción, Paraguay, who praised the initiative's revolutionary impact.\n\nThis car-free day is part of a broader urban improvement campaign started in the mid-1990s in Bogotá. The city has constructed 118 miles of bicycle paths, the most in Latin America, and has seen significant urban developments, including new parks, sports centers, improved sidewalks, reduced rush-hour traffic, and burgeoning restaurant and shopping districts, transforming the cityscape and lifestyle in Bogotá."
3,Car-free cities,The end of car culture,Elisabeth Rosenthal,https://www.nytimes.com/2013/06/30/sunday-review/the-end-of-car-culture.html,"President Obama's goals to reduce greenhouse gas emissions align with a trend in American behavior: declining car ownership and usage. Americans are buying fewer cars and driving less, with a significant drop in miles driven per person since 2005. This shift preced

In [6]:
df = pd.read_csv("input/persuade20/persuade_2.0_human_scores_demo_id_github.csv", low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25996 entries, 0 to 25995
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   essay_id_comp               25996 non-null  object 
 1   full_text                   25996 non-null  object 
 2   holistic_essay_score        25996 non-null  int64  
 3   word_count                  25996 non-null  int64  
 4   prompt_name                 25996 non-null  object 
 5   task                        25996 non-null  object 
 6   assignment                  25996 non-null  object 
 7   source_text                 12875 non-null  object 
 8   gender                      25996 non-null  object 
 9   grade_level                 24828 non-null  float64
 10  ell_status                  24787 non-null  object 
 11  race_ethnicity              25996 non-null  object 
 12  economically_disadvantaged  20759 non-null  object 
 13  student_disability_status   208

In [7]:
df.head()

,essay_id_comp,full_text,holistic_essay_score,word_count,prompt_name,task,assignment,source_text,gender,grade_level,ell_status,race_ethnicity,economically_disadvantaged,student_disability_status
0,423A1CA112E2,"Phones\n\nModern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it.\n\nWhen people have phones, they know about certain apps that they have .Apps like Facebook Twitter Instagram and Snapchat. So like if a friend moves away and you want to be in contact you can still be in contact by posting videos or text messages. People always have different ways how to communicate with a phone. Phones have changed due to our generation.\n\nDriving is one of the way how to get around. People always be on their phones while doing it. Which can cause serious Problems. That's why there's a thing that's called no texting while driving. That's a really important thing to remember. Some people still do it because they think It's stupid. No matter what they do they still have to obey it because that's the only way how did he save.\n\nSometimes on the news there is either an accident or a suicide. It might involve someone not looking where they're going or tweet that someone sent. It either injury or death. If a mysterious number says I'm going to kill you and they know where you live but you don't know the person's contact ,It makes you puzzled and make you start to freak out. Which can end up really badly.\n\nPhones are fine to use and it's also the best way to come over help. If you go through a problem and you can't find help you ,always have a phone there with you. Even though phones are used almost every day as long as you're safe it would come into use if you get into trouble. Make sure you do not be like this phone while you're in the middle of driving. The news always updated when people do something stupid around that involves their phones. The safest way is the best way to stay safe.",3,378,Phones and driving,Independent,"Today the majority of humans own and operate cell phones on a daily basis. In essay form, explain if drivers should or should not be able to use cell phones in any capacity while operating a vehicle.",NaN,M,NaN,NaN,Black/African American,NaN,NaN
1,BC75783F96E3,"This essay will explain if drivers should or should not be able to use electronic devices while operating a vehicle. Using a phone while driving can cause very bad wrecks, because they are putting themselves and other people at risk. People should not be able to use cell phones while operating a vehicle is because bad wrecks, putting others' lives in danger, and may cause death.\n\nFirst of all, wrecks can be caused by looking down at your phone while driving. Most importantly, you should always keep your eyes directly on the road when behind the wheel of a car. On that note, you will have an outrageously car payment just because you were looking down at your phone. Moreover, if you the reason the accident happened the person that was operating the other car can sue you for lots of money, and you will have to pay it. Therefore, if you don't pay whatever the person charges you consequences will be served and that's behind bars.\n\nAnother reason why we should not be able to use cell phones while operating an automobile is that you are putting other people's lives in danger. Thus, looking down at your cell phone you can get someone else hurt for the uncommunicative act you committed. As well as unconsciousness, injuries, or being in a hospital.\n\nThe main key to not texting and driving while behind the wheel of a car. Mainly, it can cause tragic, terrifying, and horrific things to you and to others and that's death. That's the mo

In [8]:
pdx.value_counts(df["prompt_name"])

,count,percent
prompt_name,,
Facial action coding system,2167,0.083359
Distance learning,2157,0.082974
Does the electoral college work?,2046,0.078704
Car-free cities,1959,0.075358
Driverless cars,1886,0.072550
Exploring Venus,1862,0.071626
Summer projects,1750,0.067318
Mandatory extracurricular activities,1670,0.064241
Cell phones at school,1656,0.063702


In [9]:
pdx.value_counts(df["task"])

,count,percent
task,,
Independent,13121,0.504731
Text dependent,12875,0.495269


In [10]:
pdx.value_counts(df["assignment"])

,count,percent
assignment,,
"In the article ""Making Mona Lisa Smile,"" the author describes how a new technology called the Facial Action Coding System enables computers to identify human emotions. Using details from the article, write an essay arguing whether the use of this technology to read the emotional expressions of students in a classroom is valuable.",2167,0.083359
Some schools offer distance learning as an option for students to attend classes from home by way of online or video conferencing. Do you think students would benefit from being able to attend classes from home? Take a position on this issue. Support your response with reasons and examples.,2157,0.082974
Write a letter to your state senator in which you argue in favor of keeping the Electoral College or changing to election by popular vote for the president of the United States. Use the information from the texts in your essay. Manage your time carefully so that you can read the passages; plan your response; write your response; and revise and edit your response. Be sure to include a claim; address counterclaims; use evidence from multiple sources; and avoid overly relying on one source. Your response should be in the form of a multiparagraph essay. Write your response in the space provided.,2046,0.078704
Write an explanatory essay to inform fellow citizens about the advantages of limiting car usage. Your essay must be based on ideas and information that can be found in the passage set. Manage your time carefully so that you can read the passages; plan your response; write your response; and revise and edit your response. Be sure to use evidence from multiple sources; and avoid overly relying on one source. Your response should be in the form of a multiparagraph essay. Write your essay in the space provided.,1959,0.075358
"In the article “Driverless Cars are Coming,” the author presents both positive and negative aspects of driverless cars. Using details from the article, create an argument for or against the development of these cars. Be sure to include: your position on driverless cars; appropriate details from the article that support your position; an introduction, a body, and a conclusion to your argumentative essay.",1886,0.072550
"In ""The Challenge of Exploring Venus,"" the author suggests studying Venus is a worthy pursuit despite the dangers it presents. Using details from the article, write an essay evaluating how well the author supports this idea. Be sure to include: a claim that evaluates how well the author supports the idea that studying Venus is a worthy pursuit despite the dangers; an explanation of the evidence from the article that supports your claim; an introduction, a body, and a conclusion to your essay.",1862,0.071626
Some schools require students to complete summer projects to assure they continue learning during their break. Should these summer projects be teacher-designed or student-designed? Take a position on this question. Support your response with reasons and specific examples.,1750,0.067318
"Your principal has decided that all students must participate in at least one extracurricular activity. For example, students could participate in sports, work on the yearbook, or serve on the student council. Do you agree or disagree with this decision? Use specific details and examples to convince others to support your position.",1670,0.064241
"Your principal is reconsidering the school's cell phone policy. She is considering two possible policies: \nPolicy 1: Allow students to bring phones to school and use them during lunch periods and other free times, as long as the phones are turned off during class time. \nPolicy 2: Do not allow students to have phones at school at all.\nWrite a letter to your principal convincing her which policy you believe is better. Support your position with specific reasons.",1656,0.063702


In [11]:
pdx.value_counts(df["source_text"])

,count,percent
source_text,,
NaN,13121,0.504731
"""Making Mona Lisa Smile"", by Nick D'Alto",2167,0.083359
"Source 1: ""What is the Electoral College?"", by the Office of the Federal Register \n Source 2: ""The Indefensible Electoral College: Why even the best-laid defenses of the system are wrong,"" by Bradford Plumer \n Source 3: ""In Defense of the Electoral College: Five reasons to keep our despised method of choosing the President,"" by Judge Richard A. Posner",2046,0.078704
"Source 1: ""In German Suburb, Life Goes On Without Cars,"" by Elisabeth Rosenthal \n Source 2: ""Paris bans driving due to smog,"" by Robert Duffer \n Source 3: ""Car-free day is spinning into a big hit in Bogota,"" by Andrew Selsky \n Source 4: ""The end of car culture,"" by Elisabeth Rosenthal",1959,0.075358
"""Driverless Cars are Coming""",1886,0.072550
"""The Challenge of Exploring Venus""",1862,0.071626
"""Unmasking the Face on Mars""",1583,0.060894
"""A Cowboy Who Rode the Waves""",1372,0.052777


In [12]:
df["clen"] = df["full_text"].str.len()
texts = df["full_text"].tolist()
x = tokenizer(texts, truncation=False, add_special_tokens=False)
df["deberta_tokens_n"] = [len(tokens) for tokens in x["input_ids"]]
df.describe(percentiles=percentiles)

,holistic_essay_score,word_count,grade_level,clen,deberta_tokens_n
count,25996.000000,25996.000000,24828.000000,25996.000000,25996.000000
mean,3.315010,418.129635,9.076808,2304.916179,470.497769
std,1.160473,275.110310,1.362257,1117.566458,214.228908
min,1.000000,146.000000,6.000000,691.000000,157.000000
1%,1.000000,158.000000,6.000000,843.000000,179.000000
5%,2.000000,181.000000,6.000000,979.000000,207.000000
10%,2.000000,205.000000,8.000000,1107.000000,234.000000
20%,2.000000,248.000000,8.000000,1349.000000,284.000000
30%,3.000000,290.000000,8.000000,1581.000000,331.000000
40%,3.000000,331.000000,8.000000,1822.000000,379.000000


In [13]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:00:17.737309
